In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')


# Change directory to the location of the file
%cd /content/drive/Shareddrives/ACS6403 - Group Project/Deliverable 5 Final Project Report/Prediction Model/

input_data_1 = '/content/drive/Shareddrives/ACS6403 - Group Project/Preprocessed Data/input_data.txt'  # Update the path
output_data_1 = '/content/drive/Shareddrives/ACS6403 - Group Project/Preprocessed Data/output_data.txt'  # Update the path


# Load the input and output data
input_Data = pd.read_csv(input_data_1, sep=',')
input_Data.columns = ['Input1', 'Input2', 'Input3']

output_Data = pd.read_csv(output_data_1, sep='\s+')
output_Data.columns = ['Output1', 'Output2', 'Output3', 'Output4', 'Output5', 'Output6', 'Output7']

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale input and output data
input_Data_normalized = scaler.fit_transform(input_Data)
input_Data_normalized = pd.DataFrame(input_Data_normalized, columns=input_Data.columns)

output_Data_normalized = scaler.fit_transform(output_Data)
output_Data_normalized = pd.DataFrame(output_Data_normalized, columns=output_Data.columns)
# # Save the normalized input data to a text file with comma-separated values
input_Data_normalized.to_csv('input_data_normalized.txt', sep=',', index=False)

# # Save the normalized output data to a text file with comma-separated values
output_Data_normalized.to_csv('output_data_normalized.txt', sep=',', index=False)


print(input_Data_normalized)
print(output_Data_normalized)

# # Save the normalized input data to a text file with comma-separated values
input_Data_normalized.to_csv('input_data_normalized.txt', sep=',', index=False)

# # Save the normalized output data to a text file with comma-separated values
output_Data_normalized.to_csv('output_data_normalized.txt', sep=',', index=False)

# Open the text file for reading
# Open the text file for reading
with open('input_data_normalized.txt', 'r') as input_file:
    # Open a CSV file for writing
    with open('input_data_normalized.csv', 'w') as output_file:
        # Read line by line from the text file
        for line in input_file:
            # Replace space with comma
            line = line.strip()

            # Write the modified line to the CSV file
            output_file.write(line + '\n')
with open('output_data_normalized.txt', 'r') as input_file:
    # Open a CSV file for writing
    with open('output_data_normalized.csv', 'w') as output_file:
        # Read line by line from the text file
        for line in input_file:
            # Replace space with comma
            line = line.strip().replace(' ', ',')

            # Write the modified line to the CSV file
            output_file.write(line + '\n')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/ACS6403 - Group Project/Deliverable 5 Final Project Report/Prediction Model
       Input1  Input2    Input3
0         0.0     0.0  0.020408
1         0.0     0.0  0.040816
2         0.0     0.0  0.061224
3         0.0     0.0  0.081633
4         0.0     0.0  0.102041
...       ...     ...       ...
74994     1.0     1.0  0.918367
74995     1.0     1.0  0.938776
74996     1.0     1.0  0.959184
74997     1.0     1.0  0.979592
74998     1.0     1.0  1.000000

[74999 rows x 3 columns]
        Output1   Output2   Output3       Output4       Output5       Output6  \
0      0.767588  0.999854  0.910629  1.317981e-12  1.857436e-16  4.774623e-16   
1      0.767830  0.999409  0.925470  1.317981e-12  4.664888e-16  1.083256e-15   
2      0.768068  0.998957  0.934249  1.317981e-12  7.659854e-16  1.850238e-15   
3      0.768302  0.998497  0.9405

In [12]:
import numpy as np
import pandas as pd
from scipy.signal import lsim
from scipy.optimize import least_squares
import statsmodels.api as sm
from sklearn.model_selection import KFold
import joblib

# Calculating the start time
import time
tstart = time.process_time()

# Loading the input and output data
input_data2 = pd.read_csv('input_data_normalized.csv')
output_data2 = pd.read_csv('output_data_normalized.csv')

# Generating the models
models = []

for i in range(7):
    output_data = output_data2.iloc[:, i]  # Selecting the i-th column of output_data2
    input_data = input_data2.copy()  # Making a copy of input_data2 to avoid modifying the original DataFrame
    input_data[f'Input{4}'] = output_data # Setting the 4th column of input_data to the selected column of output_data

    # Simulated data
    np.random.seed(0)
    nobs = 74999
    # Simulating two input variables and two output variables
    output_data = np.dot(input_data, np.array([[0.5, 0.3], [0.2, 0.8],[0.2,0.1],[0.4,0.8]])) + np.random.randn(nobs, 2)

    # VAR model specification
    model = sm.tsa.VAR(output_data)  # Assuming output_data is a DataFrame or array of time series data
    results = model.fit()  # maxlags is the maximum lag order for the VAR model

    # Append the model to the list of models
    models.append(results)

    # Save each model as a joblib file
    joblib.dump(results, f'Copy of System_model_{i}.joblib')
    print(f"Model {i+1} saved as VAR_model_{i}.joblib")

    # VARMA results
    print(results.summary())

mae_values = []  # List to store MAE values for each model

for i, model in enumerate(models):
    # Predicting the output using the VAR model
    lag_order = model.k_ar  # Lag order used in the VAR model
    predicted_data = model.forecast(output_data, steps=lag_order)  # Forecasting next 'lag_order' steps

    # Calculating MAE
    absolute_errors = np.abs(predicted_data - output_data[lag_order:])  # Calculate absolute errors
    mean_absolute_error = np.mean(absolute_errors)  # Calculate mean absolute error
    mae_values.append(mean_absolute_error)

    print(f"Model {i+1} MAE: {mean_absolute_error}")

# Overall mean MAE
overall_mae = np.mean(mae_values)
print("Overall Mean Absolute Error:", overall_mae)

# Define the number of folds for cross-validation
k_folds = 5

# Initialize lists to store performance metrics for each model
mae_values = []
overall_mae = []

# Initialize the KFold object
kf = KFold(n_splits=k_folds)

# Loop through each fold
for train_index, test_index in kf.split(input_data2):
    # Split data into training and testing sets
    input_train, input_test = input_data2.iloc[train_index], input_data2.iloc[test_index]
    output_train, output_test = output_data2.iloc[train_index], output_data2.iloc[test_index]

    # Train VAR model on the training set
    model = sm.tsa.VAR(output_train)
    results = model.fit()

    # Forecast output using VAR model
    lag_order = results.k_ar
    predicted_data = results.forecast(output_train.values[-lag_order:], len(test_index))

    # Calculate MAE for this fold
    absolute_errors = np.abs(predicted_data - output_test.values)
    fold_mae = np.mean(absolute_errors)

    # Store fold MAE
    mae_values.append(fold_mae)

    # Store model results for later reference
    models.append(results)

# Calculate overall MAE across all folds
overall_mae = np.mean(mae_values)

# Find the best model based on the lowest MAE
best_model_index = np.argmin(mae_values)
best_model = models[best_model_index]

# Print the best model's summary
print("Best Model Summary:")
print(best_model.summary())

# Print overall mean MAE
print("Overall Mean Absolute Error:", overall_mae)

# Calculating the end time
tend = time.process_time() - tstart
print("Time taken:", tend, "seconds")


Model 1 saved as VAR_model_0.joblib
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 13, May, 2024
Time:                     03:34:58
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                  0.0799894
Nobs:                     74998.0    HQIC:                 0.0794783
Log likelihood:          -215801.    FPE:                    1.08248
AIC:                    0.0792514    Det(Omega_mle):         1.08239
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.728308         0.006331          115.039           0.000
L1.y1         0.017995         0.003652            4.928           0.000
L1.y2         0.020031         0.003558            5.629   

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Best Model Summary:
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 13, May, 2024
Time:                     03:35:01
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   0.103017
Nobs:                     74998.0    HQIC:                  0.102506
Log likelihood:          -216664.    FPE:                    1.10769
AIC:                     0.102279    Det(Omega_mle):         1.10760
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.427231         0.004648           91.920           0.000
L1.y1         0.035463         0.003651            9.714           0.000
L1.y2         0.039477         0.003575           11.042           0.000

R